<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Two: Feature Selection </h2>	


<h4> In this section, we would like you to select between 15 and 20 features to focus your model on. This will require significant explatoratory research. The first one is already implemented for you, and the next two are pre-specified.  </h4>

In [6]:
import pandas as pd
import numpy as np
import skimage.io as io
import cv2 as cv
starting_data = pd.read_pickle("./starting_data.pkl")

ModuleNotFoundError: No module named 'cv2'

In [4]:
# def get_intensity_based_features(image):
#     avg_channel = np.zeros(3)
#     for i in range(3):
#         avg_channel[i] = np.mean(image[:,:,i] )
#     a=np.argmax(image, axis=2)
#     ind, counts = np.unique(a, return_counts=True)
#     percent_max = counts/np.sum(counts)
#     return [avg_channel, percent_max]

# def getImageFeatures(image):
#     # handle condition when images are black-white
#     # by copying b/w image to all three channels
    
#     if len(image.shape) == 2:
#         temp = np.zeros((image.shape[0], image.shape[1], 3))
#         for i in range(3):
#             temp[:,:,i] = image
#         image = temp
    
#     m,n,k = image.shape
    
#     # Get full image features
#     full_image_features= get_intensity_based_features(image)
    
#     patch_min = int(m/4)
#     patch_max = int(3*m/4)
#     cropped = image[patch_min:patch_max, patch_min:patch_max, :]
    
#     # Get central cropped image features
#     cropped_image_features = get_intensity_based_features(cropped)
    
    
#     return [full_image_features, cropped_image_features] 

def normalize(image):
    if len(image.shape) == 2:
        temp = np.zeros((image.shape[0], image.shape[1], 3))
        for i in range(3):
            temp[:,:,i] = image
        image = temp
    return image

# Returns the average of the red-channel pictures for the images
def red_avg(image):
    return np.mean(image[:, :, 0])

# Returns the average of the green-channel pictures for the images
def green_avg(image):
    return np.mean(image[:, :, 1])

# Returns the average of the blue-channel pictures for the images
def blue_avg(image):
    return np.mean(image[:, :, 2])

def argmax(image):
    orig = np.zeros(3)
    a = np.argmax(image, axis = 2)
    vals, counts = np.unique(a, return_counts=True)
    for i in range(3):
        if i in vals:
            orig[i] = counts[np.where(vals == i)]
    percent_max = orig/np.sum(orig)
    if len(percent_max) != 3:
        print('Help! Percent max length is not 3')        
    return percent_max

def crop(image):
    m, n, k = image.shape
    patch_min = int(m/4)
    patch_max = int(3*m/4)
    cropped = image[patch_min:patch_max, patch_min:patch_max, :]    
    return cropped

def contrast(image):
    vals = np.zeros(3)
    for i in range(3):
        vals[i] = (np.max(image[:, :, i]) - np.min(image[:, :, i]))
    return np.mean(vals)

def getImageFeatures(df):
    df["image"] = df["image"].apply(normalize)
    df["red"] = df["image"].apply(red_avg)
    df["green"] = df["image"].apply(green_avg)
    df["blue"] = df["image"].apply(blue_avg)
    df["argmax"] = df["image"].apply(argmax)
    df[["argmax_r","argmax_g","argmax_b"]] = pd.DataFrame(df.argmax.values.tolist(), index= df.index)
    df["contrast"] = df["image"].apply(contrast)
    
    df["cropped"] = df["image"].apply(crop)
    df["red_crop"] = df["cropped"].apply(red_avg)
    df["green_crop"] = df["cropped"].apply(green_avg)
    df["blue_crop"] = df["cropped"].apply(blue_avg)
    df["argmax_crop"] = df["cropped"].apply(argmax)
    df[["argmax__crop_r","argmax_crop_g","argmax_crop_b"]] = pd.DataFrame(df.argmax_crop.values.tolist(), index= df.index)
    df["contrast_crop"] = df["cropped"].apply(contrast)

    #Uncomment the following line when ready to make the final dataframe for Part 3!
    df = df.drop(labels = ['image', 'filename', 'cropped','argmax', 'argmax_crop'], axis = 1)
    
    return df

starting_data = getImageFeatures(starting_data)
starting_data.head()
#starting_data['argmax'].str.split(',', expand = True)

,class,aspect,argmax_r,argmax_g,argmax_b,red,green,blue,contrast,red_crop,green_crop,blue_crop,argmax__crop_r,argmax_crop_g,argmax_crop_b,contrast_crop
0,0,2.426829,0.572327,0.421021,0.006653,0.719047,0.693171,0.585263,0.997897,0.631888,0.629310,0.537280,0.536377,0.460449,0.003174,0.982738
1,0,2.179348,0.971802,0.028198,0.000000,0.826595,0.742442,0.646464,0.979614,0.675802,0.498426,0.410969,0.984375,0.015625,0.000000,0.940746
2,0,2.381818,0.705322,0.056702,0.237976,0.666337,0.578738,0.437090,0.993714,0.611749,0.418503,0.193338,0.809326,0.065430,0.125244,0.864494
3,0,2.311765,0.835449,0.161743,0.002808,0.597665,0.521160,0.306170,0.994251,0.575981,0.407758,0.148851,0.943848,0.052734,0.003418,0.969045
4,0,2.244318,0.404968,0.515442,0.079590,0.576913,0.589097,0.337259,0.998115,0.393732,0.401980,0.183056,0.262695,0.493408,0.243896,0.994631


In [ ]:
def edges(image):
    image = io.imread(image)
    
    m,n,k = image.shape
    
edges_bear = edges(r"data/20_categories_training/bear/bear_0011.jpg")


In [5]:
starting_data.shape

(1501, 16)

Define more features above, performing any EDA research below. We expect all external sources sited, and a couple significant different graphs indicating some form of EDA. 

<h4> Graphs </h4>

In [4]:
# store -r starting_data

<h4> Sources </h4>

In [5]:
files=[]
for i,image in enumerate(starting_data["image"]):
    if len(image.shape) < 3:
        files.append(starting_data.iloc[i, 2])
print(files)

['blimp_0022.jpg', 'comet_0006.jpg', 'comet_0011.jpg', 'comet_0013.jpg', 'comet_0021.jpg', 'comet_0036.jpg', 'comet_0038.jpg', 'comet_0041.jpg', 'comet_0049.jpg', 'comet_0052.jpg', 'comet_0053.jpg', 'comet_0057.jpg', 'comet_0058.jpg', 'crab_0045.jpg', 'dolphin_0025.jpg', 'gorilla_0128.jpg']


<h4> DataFrame Creation </h4>

In [6]:
def feature_frame(df):
    return df
    #Returns data-frame with all the features now inside, and calculated


In [ ]:
feature_frame(data_from_nb1)